In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

import time

In [9]:
epochs = 5
batch_sz = 64
learning_rate = 1

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size= 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size= 5)
        self.dropout = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320, 100)
        self.fc2 = nn.Linear(100,10)   
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(F.dropout2d(self.conv2(x)), 2))
        
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [11]:
train_dataset = dsets.MNIST(root = './data', 
                             train= True,
                             download=True, 
                             transform= transforms.ToTensor())

test_dataset = dsets.MNIST(root= './data', 
                            train = False,
                            transform= transforms.ToTensor())

In [12]:
train_loader = torch.utils.data.DataLoader(batch_size= batch_sz,
                                           dataset= train_dataset, 
                                           shuffle= True)

test_loader = torch.utils.data.DataLoader(batch_size= batch_sz, 
                                          dataset = test_dataset, 
                                          shuffle = False)

In [13]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [15]:
startTime = time.time()

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()

endTime = time.time()
print("Time elapsed to train = " + str(endTime - startTime))


Time elapsed = 125.32634782791138


In [29]:
i=0
correct = 0
total = 0
for images, labels in (test_loader):
    images = Variable(images)
    output = net(images)
    _, pred = torch.max(output.data, 1)
    correct += (labels == pred).sum().item()
    total += len(pred)
print('Accuracy of the network on the 10K test images: %.2f %%' % (100 * correct / float(total) ))

Accuracy of the network on the 10K test images: 94.98 %
